# Import Dataset & Librairies

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
flights_sample_train= pd.read_csv('../../data/processed/Enriched-flights_sample_train.csv') # Used for training model
flights_sample_test= pd.read_csv('../../data/processed/Enriched-flights_sample_test.csv') # Used for testing model

flights_test= pd.read_csv('../../data/processed/Enriched-flights_BIGtest(2dec).csv') # Simulation of the test data set. Full random 200K
flights_final_test= pd.read_csv('../../data/processed/Enriched-flights_test.csv') # Mid_term Project Test dataset

C:\Users\987\AppData\Local\Temp\ipykernel_39484\3081880053.py:5: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_final_test= pd.read_csv('../../data/processed/Enriched-flights_test.csv') # Mid_term Project Test dataset


## XGBoost Regressor RMSE: 20.93, 45.98, 48.64 Submitted: Yes

### Train Model

In [73]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 20.933387


### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [15]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [16]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 45.989178


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [76]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [75]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']


## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [77]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 48.646433


#### Export 2nd Test Results to CSV (Final QC)

In [78]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/XGBoostReg.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [20]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

{'cancelled', 'Weather Delay (minutes)', 'Operator - Unique Carrier Code2_9K', 'diverted', 'Carrier Delay (minutes)', 'Operator - Unique Carrier Code2_KS', 'National Air System Delay (minutes)', 'Operator - Unique Carrier Code2_VX', 'Aircraft type2_432.0', 'Security Delay (minutes)', 'Departure Delay (minutes)', 'cancellation_code', 'Arrival Delay (minutes)', 'Late Aircraft Delay (minutes)'}

set()


In [21]:
## Add missing columns to the test (1st set) ... Most of those will get dropped right after anyway.. it's really just so the arrays match
flights_final_test['cancelled'] = 0
flights_final_test['Weather Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_9K'] = 0
flights_final_test['diverted'] = 0
flights_final_test['Carrier Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_KS'] = 0
flights_final_test['National Air System Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_VX'] = 0
flights_final_test['Aircraft type2_432.0'] = 0
flights_final_test['Security Delay (minutes)'] = 0
flights_final_test['Departure Delay (minutes)'] = 0
flights_final_test['cancellation_code'] = 0
flights_final_test['Arrival Delay (minutes)'] = 0
flights_final_test['Late Aircraft Delay (minutes)'] = 0

## Remove additional columns from the test (2nd set)



## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [24]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

In [26]:
# output y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)

# Rename the columns
y_pred_df.columns=['predicted_delay']

# Now, we have to format y_pred_df to match the format of the sample submission file
## Produce the missing columns
flights_test['fl_date'] = flights_test['Flight Year'].astype(str) + '-' + flights_test['Flight Month'].astype(str).str.zfill(2) + '-' + flights_test['Flight Day'].astype(str).str.zfill(2)
flights_test['mkt_carrier'] = flights_test['Marketer - Unique Carrier Code']
flights_test['mkt_carrier_fl_num'] = flights_test['Flight Number']
flights_test['origin'] = flights_test['Origin Airport (IATA Code)']
flights_test['dest'] = flights_test['Destination Airport (IATA Code)']

## Add these columns to y_pred_df
y_pred_df['fl_date'] = flights_test['fl_date']
y_pred_df['mkt_carrier'] = flights_test['mkt_carrier']
y_pred_df['mkt_carrier_fl_num'] = flights_test['mkt_carrier_fl_num']
y_pred_df['origin'] = flights_test['origin']
y_pred_df['dest'] = flights_test['dest']

## Reorder the columns to match the sample submission file
y_pred_df = y_pred_df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']]

# Export to csv
y_pred_df.to_csv('../../output/XGBoostReg.csv', index=False)

## XGBoost Regressor - What if we added more data? RMSE (40.00, 46.05)

Let's add our 200K to our training data and see if we can better predict the 40K

#### Let's start by making sure both have the same columns and are in the same order

In [90]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()
set()


In [91]:
## Adding the missing column to the test
#flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

In [92]:
flights_sample_combined_train = pd.concat([flights_sample_train, flights_test], axis=0)

### Train the model

In [93]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_combined_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_combined_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 40.002790


So this is not really helping our initial value, but what about the 40K?

### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [94]:
print(set(flights_sample_combined_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_combined_train.columns))

set()
set()


#### Now run the test

In [95]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 46.057570


Nope.. this got worse..

### Not a tuning per say, but what is the accuracy score of XGBoost Reg?

## XGBoost Regressor - Tuning the features (dropped)

### Let's try keeping only our correlated features  RMSE: 21.20

In [80]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
y=flights_sample_train['Arrival Delay (minutes)']

#Let's try keeping only our correlated features
X = X[X.columns[(X.corrwith(y) > 0.01).tolist()]]

X=X.fillna(0)
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 21.209647


Well that's not really doing any better...

### How about Min/Max Scaler?  RMSE: 20.93

In [82]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
y=flights_sample_train['Arrival Delay (minutes)']

X=X.fillna(0)
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# Scaling the input features using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=seed)

# fit model to training data
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 20.933513


Not doing anything

## Random Forest Regressor   RMSE: 20.681228, 45.845254, 48.44 Submitted: Yes

### Train Model

In [48]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 20.681228


### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [28]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [49]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 45.845254


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [53]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [52]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']

## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [56]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 48.447882


#### Export 2nd Test Results to CSV (Final QC)

In [57]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/RandomForestReg.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [60]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

set()

set()


In [59]:
## Add missing columns to the test (1st set) ... Most of those will get dropped right after anyway.. it's really just so the arrays match
flights_final_test['cancelled'] = 0
flights_final_test['Weather Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_9K'] = 0
flights_final_test['diverted'] = 0
flights_final_test['Carrier Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_KS'] = 0
flights_final_test['National Air System Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_VX'] = 0
flights_final_test['Aircraft type2_432.0'] = 0
flights_final_test['Security Delay (minutes)'] = 0
flights_final_test['Departure Delay (minutes)'] = 0
flights_final_test['cancellation_code'] = 0
flights_final_test['Arrival Delay (minutes)'] = 0
flights_final_test['Late Aircraft Delay (minutes)'] = 0

## Remove additional columns from the test (2nd set)



## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [61]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

In [62]:
# output y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)

# Rename the columns
y_pred_df.columns=['predicted_delay']

# Now, we have to format y_pred_df to match the format of the sample submission file
## Produce the missing columns
flights_test['fl_date'] = flights_test['Flight Year'].astype(str) + '-' + flights_test['Flight Month'].astype(str).str.zfill(2) + '-' + flights_test['Flight Day'].astype(str).str.zfill(2)
flights_test['mkt_carrier'] = flights_test['Marketer - Unique Carrier Code']
flights_test['mkt_carrier_fl_num'] = flights_test['Flight Number']
flights_test['origin'] = flights_test['Origin Airport (IATA Code)']
flights_test['dest'] = flights_test['Destination Airport (IATA Code)']

## Add these columns to y_pred_df
y_pred_df['fl_date'] = flights_test['fl_date']
y_pred_df['mkt_carrier'] = flights_test['mkt_carrier']
y_pred_df['mkt_carrier_fl_num'] = flights_test['mkt_carrier_fl_num']
y_pred_df['origin'] = flights_test['origin']
y_pred_df['dest'] = flights_test['dest']

## Reorder the columns to match the sample submission file
y_pred_df = y_pred_df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']]

# Export to csv
y_pred_df.to_csv('../../output/RandomForestReg.csv', index=False)

## SVM RMSE:21.86, 47.77, ?  Export:No...........Takes too long!

SVM is extremely slow... so far we haven't been able to finish running the models

### Train Model

In [36]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import sklearn
from sklearn import svm

# model = svm.SVR()
# model.fit(X_train, y_train)
model = pickle.load(open('../SVMmodel.pkl', 'rb')) # Using the pickle'd version of the trained model

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_test, y_pred))
print(rms)

21.86935551774908


### Save model into a pickle

### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [34]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [35]:
## Adding the missing column to the test
#flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [37]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 47.775416


### Second Test (200K) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [41]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [42]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']


## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [ ]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

model = pickle.load(open('../pickles/SVMmodel.pkl', 'rb'))

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

#### Export 2nd Test Results to CSV (Final QC)

In [ ]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/SVM.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [41]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

set()

set()


In [42]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']


## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [ ]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values


model = pickle.load(open('../pickles/SVMmodel.pkl', 'rb'))

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

## Random Forest Classifier   RMSE: 24.62, 49.42, 51.80  Submitted: Yes

### Train Model

In [63]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 24.645421


### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [64]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [65]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 49.370359


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [66]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

{'origin', 'mkt_carrier', 'dest', 'mkt_carrier_fl_num', 'fl_date'}


In [67]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']

## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [68]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 51.899348


#### Export 2nd Test Results to CSV (Final QC)

In [69]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/RandomForestCl.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [70]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

set()

set()


In [21]:
## Add missing columns to the test (1st set) ... Most of those will get dropped right after anyway.. it's really just so the arrays match
flights_final_test['cancelled'] = 0
flights_final_test['Weather Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_9K'] = 0
flights_final_test['diverted'] = 0
flights_final_test['Carrier Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_KS'] = 0
flights_final_test['National Air System Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_VX'] = 0
flights_final_test['Aircraft type2_432.0'] = 0
flights_final_test['Security Delay (minutes)'] = 0
flights_final_test['Departure Delay (minutes)'] = 0
flights_final_test['cancellation_code'] = 0
flights_final_test['Arrival Delay (minutes)'] = 0
flights_final_test['Late Aircraft Delay (minutes)'] = 0

## Remove additional columns from the test (2nd set)



## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [71]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

In [72]:
# output y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)

# Rename the columns
y_pred_df.columns=['predicted_delay']

# Now, we have to format y_pred_df to match the format of the sample submission file
## Produce the missing columns
flights_test['fl_date'] = flights_test['Flight Year'].astype(str) + '-' + flights_test['Flight Month'].astype(str).str.zfill(2) + '-' + flights_test['Flight Day'].astype(str).str.zfill(2)
flights_test['mkt_carrier'] = flights_test['Marketer - Unique Carrier Code']
flights_test['mkt_carrier_fl_num'] = flights_test['Flight Number']
flights_test['origin'] = flights_test['Origin Airport (IATA Code)']
flights_test['dest'] = flights_test['Destination Airport (IATA Code)']

## Add these columns to y_pred_df
y_pred_df['fl_date'] = flights_test['fl_date']
y_pred_df['mkt_carrier'] = flights_test['mkt_carrier']
y_pred_df['mkt_carrier_fl_num'] = flights_test['mkt_carrier_fl_num']
y_pred_df['origin'] = flights_test['origin']
y_pred_df['dest'] = flights_test['dest']

## Reorder the columns to match the sample submission file
y_pred_df = y_pred_df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']]

# Export to csv
y_pred_df.to_csv('../../output/RandomForestCl.csv', index=False)

## Linear Regression  RMSE: 994140.93, 2435111.90, 2037559.82  Submitted: No

### Train Model

In [33]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 994140.935793


### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [28]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [34]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 2435111.903582


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [31]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [30]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']

## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [35]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 2037559.822769


#### Export 2nd Test Results to CSV (Final QC)

In [36]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/LinReg.csv', index=False)

### Final Test (Jan2020) - Not submitting this one results are too bad

## XG Boost Regressor - Error at training

### Train Model

In [27]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import xgboost as xgb
from xgboost import XGBRegressor

model = XGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149], got [-80. -73. -67. -64. -62. -59. -58. -56. -55. -54. -53. -52. -51. -50.
 -49. -48. -47. -46. -45. -44. -43. -42. -41. -40. -39. -38. -37. -36.
 -35. -34. -33. -32. -31. -30. -29. -28. -27. -26. -25. -24. -23. -22.
 -21. -20. -19. -18. -17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.
  -7.  -6.  -5.  -4.  -3. 

### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [28]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [16]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 45.989178


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [31]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [30]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']

## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [18]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 48.646433


#### Export 2nd Test Results to CSV (Final QC)

In [19]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/XGBoostReg.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [32]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

set()

set()


In [21]:
## Add missing columns to the test (1st set) ... Most of those will get dropped right after anyway.. it's really just so the arrays match
flights_final_test['cancelled'] = 0
flights_final_test['Weather Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_9K'] = 0
flights_final_test['diverted'] = 0
flights_final_test['Carrier Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_KS'] = 0
flights_final_test['National Air System Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_VX'] = 0
flights_final_test['Aircraft type2_432.0'] = 0
flights_final_test['Security Delay (minutes)'] = 0
flights_final_test['Departure Delay (minutes)'] = 0
flights_final_test['cancellation_code'] = 0
flights_final_test['Arrival Delay (minutes)'] = 0
flights_final_test['Late Aircraft Delay (minutes)'] = 0

## Remove additional columns from the test (2nd set)



## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [24]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

In [26]:
# output y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)

# Rename the columns
y_pred_df.columns=['predicted_delay']

# Now, we have to format y_pred_df to match the format of the sample submission file
## Produce the missing columns
flights_test['fl_date'] = flights_test['Flight Year'].astype(str) + '-' + flights_test['Flight Month'].astype(str).str.zfill(2) + '-' + flights_test['Flight Day'].astype(str).str.zfill(2)
flights_test['mkt_carrier'] = flights_test['Marketer - Unique Carrier Code']
flights_test['mkt_carrier_fl_num'] = flights_test['Flight Number']
flights_test['origin'] = flights_test['Origin Airport (IATA Code)']
flights_test['dest'] = flights_test['Destination Airport (IATA Code)']

## Add these columns to y_pred_df
y_pred_df['fl_date'] = flights_test['fl_date']
y_pred_df['mkt_carrier'] = flights_test['mkt_carrier']
y_pred_df['mkt_carrier_fl_num'] = flights_test['mkt_carrier_fl_num']
y_pred_df['origin'] = flights_test['origin']
y_pred_df['dest'] = flights_test['dest']

## Reorder the columns to match the sample submission file
y_pred_df = y_pred_df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']]

# Export to csv
y_pred_df.to_csv('../../output/XGBoostReg.csv', index=False)

## OLS - Error at training

### Train Model

In [37]:
# Let's set X and y to our flights_sample_train data and 'Arriaval Delay (minutes)' column
X=flights_sample_train.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_train['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# split into train and test sets
seed = 1
test_size = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# fit model to training data
import statsmodels.api as sm

model = sm.OLS
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

C:\Users\987\anaconda3\envs\main_ide\lib\site-packages\statsmodels\regression\linear_model.py:317: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if method == "pinv":
C:\Users\987\anaconda3\envs\main_ide\lib\site-packages\statsmodels\regression\linear_model.py:332: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif method == "qr":


ValueError: method has to be "pinv" or "qr"

### First Test (40K)

#### Let's start by making sure both have the same columns and are in the same order

In [28]:
print(set(flights_sample_train.columns) - set(flights_sample_test.columns))
print(set(flights_sample_test.columns) - set(flights_sample_train.columns))

set()
set()


In [9]:
## Adding the missing column to the test
flights_sample_test['Aircraft type2_887.0'] = 0

## Not let's put our columns in the same order:
flights_sample_test = flights_sample_test[flights_sample_train.columns]

#### Now run the test

In [16]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_sample_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_sample_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 45.989178


Well that's not good..

### Second Test (200K)

#### Let's start by making sure both have the same columns and are in the same order

In [31]:
print(set(flights_sample_train.columns) - set(flights_test.columns))
print()
print(set(flights_test.columns) - set(flights_sample_train.columns))

set()

set()


In [30]:
## Add missing columns to the test (1st set)

## Remove additional columns from the test (2nd set)
#del flights_test['Aircraft type2_837.0']
del flights_test['origin']
del flights_test['mkt_carrier']
del flights_test['dest']
del flights_test['mkt_carrier_fl_num']
del flights_test['fl_date']

## Now let's put our columns in the same order:
flights_test = flights_test[flights_sample_train.columns]

#### Now run the test

In [18]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)
y=flights_test['Arrival Delay (minutes)']
y=y.fillna(0)

# convert to numpy array
X=X.values
y=y.values

# make predictions for test data
y_pred = model.predict(X)

# evaluate predictions
rmse = np.sqrt(mean_squared_error(y, y_pred))
print("RMSE: %f" % (rmse))

RMSE: 48.646433


#### Export 2nd Test Results to CSV (Final QC)

In [19]:
# output y and y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)
y_test_df=pd.DataFrame(y)

# merge the two dataframes
y_pred_df['y_test']=y_test_df

# rename the columns
y_pred_df.columns=['Model Prediction', 'Actual Value']

#export to csv
y_pred_df.to_csv('../../src/modules/2nd Test Predictions/OLS.csv', index=False)

### Final Test (Jan2020) - Incomplete

#### Let's start by making sure both have the same columns and are in the same order

In [32]:
print(set(flights_sample_train.columns) - set(flights_final_test.columns))
print()
print(set(flights_final_test.columns) - set(flights_sample_train.columns))

set()

set()


In [21]:
## Add missing columns to the test (1st set) ... Most of those will get dropped right after anyway.. it's really just so the arrays match
flights_final_test['cancelled'] = 0
flights_final_test['Weather Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_9K'] = 0
flights_final_test['diverted'] = 0
flights_final_test['Carrier Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_KS'] = 0
flights_final_test['National Air System Delay (minutes)'] = 0
flights_final_test['Operator - Unique Carrier Code2_VX'] = 0
flights_final_test['Aircraft type2_432.0'] = 0
flights_final_test['Security Delay (minutes)'] = 0
flights_final_test['Departure Delay (minutes)'] = 0
flights_final_test['cancellation_code'] = 0
flights_final_test['Arrival Delay (minutes)'] = 0
flights_final_test['Late Aircraft Delay (minutes)'] = 0

## Remove additional columns from the test (2nd set)



## Now let's put our columns in the same order:
flights_final_test = flights_final_test[flights_sample_train.columns]

#### Now run the test.. no score for this one, we're just predicting!

In [24]:
# Now let's try to predict the arrival delay for the flights_sample_test data
X=flights_final_test.drop(['Marketer - Unique Carrier Code', 'Operator - Unique Carrier Code', 'Tail Number', 'Flight Number', 'Aircraft group', 'Aircraft type', 'Aircraft configuration', 'Service class', 'Distance (miles)', 'Origin Airport (IATA Code)', 'Airport Type_origin', 'Scheduled Departure Time (local time)', 'Destination Airport (IATA Code)', 'Airport Type_destination', 'Scheduled Arrival Time (local time)', 'Departure Delay (minutes)', 'Arrival Delay (minutes)', 'Carrier Delay (minutes)', 'Weather Delay (minutes)', 'National Air System Delay (minutes)', 'Security Delay (minutes)', 'Late Aircraft Delay (minutes)', 'cancelled', 'cancellation_code', 'diverted'], axis=1)
X=X.fillna(0)

# convert to numpy array
X=X.values

# make predictions for test data
y_pred = model.predict(X)

### Export Final_Test Results to CSV

In [26]:
# output y pred into a dataframe
y_pred_df=pd.DataFrame(y_pred)

# Rename the columns
y_pred_df.columns=['predicted_delay']

# Now, we have to format y_pred_df to match the format of the sample submission file
## Produce the missing columns
flights_test['fl_date'] = flights_test['Flight Year'].astype(str) + '-' + flights_test['Flight Month'].astype(str).str.zfill(2) + '-' + flights_test['Flight Day'].astype(str).str.zfill(2)
flights_test['mkt_carrier'] = flights_test['Marketer - Unique Carrier Code']
flights_test['mkt_carrier_fl_num'] = flights_test['Flight Number']
flights_test['origin'] = flights_test['Origin Airport (IATA Code)']
flights_test['dest'] = flights_test['Destination Airport (IATA Code)']

## Add these columns to y_pred_df
y_pred_df['fl_date'] = flights_test['fl_date']
y_pred_df['mkt_carrier'] = flights_test['mkt_carrier']
y_pred_df['mkt_carrier_fl_num'] = flights_test['mkt_carrier_fl_num']
y_pred_df['origin'] = flights_test['origin']
y_pred_df['dest'] = flights_test['dest']

## Reorder the columns to match the sample submission file
y_pred_df = y_pred_df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']]

# Export to csv
y_pred_df.to_csv('../../output/OLS.csv', index=False)

##